In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Imports

import numpy as np
import pandas as pd
import time

In [ ]:
# Get train dataset and test dataset
trainset = pd.read_csv("q3_train_dataset.csv")
testset = pd.read_csv("q3_test_dataset.csv")

# Separate labels from trainset and testset
trainlabels = trainset["Survival Status"].astype(int)
testlabels = testset["Survival Status"].astype(int)

# Get features
trainset = trainset[["Ticket Class", "Gender", "Age", "Siblings / Spouse", "Parent / Children", "Port of Embarkation"]]
testset = testset[["Ticket Class", "Gender", "Age", "Siblings / Spouse", "Parent / Children", "Port of Embarkation"]]

# Put 0 for male and 1 for female
trainset["Gender"].replace({"male":0, "female":1}, inplace=True)
testset["Gender"].replace({"male":0, "female":1}, inplace=True)

# Put 0 for S, 1 for C, 2 for Q
trainset["Port of Embarkation"].replace({"S":0, "C":1, "Q":2}, inplace=True)
testset["Port of Embarkation"].replace({"S":0, "C":1, "Q":2}, inplace=True)

# Convert float ages to integers (Maybe not needed)
trainset["Age"] = trainset["Age"].astype(int)
testset["Age"] = testset["Age"].astype(int)

# Need to normalize data
normalizedTrainset=(trainset-trainset.min())/(trainset.max()-trainset.min())
normalizedTestset = (testset-testset.min())/(testset.max()-testset.min())

# Convert trainset and testset and labels into numpy arrays
trainset = np.asarray(normalizedTrainset)
testset = np.asarray(normalizedTestset)

trainlabels = np.asarray(trainlabels)
testlabels = np.asarray(testlabels)

In [ ]:
# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def predict(X, W, b):
    return np.round(sigmoid(np.dot(X,W) + b))

In [ ]:
def accuracy(cm):
    tp = cm[0,0]
    tn = cm[1,1]
    fp = cm[1,0]
    fn = cm[0,1]

    return (tp + tn)/(tp+tn+fp+fn)
def precision(cm):
    tp = cm[0,0]
    tn = cm[1,1]
    fp = cm[1,0]
    fn = cm[0,1]

    return tp/(tp+fp)
def recall(cm):
    tp = cm[0,0]
    tn = cm[1,1]
    fp = cm[1,0]
    fn = cm[0,1]

    return tp/(tp+fn)
def NPV(cm):
    tp = cm[0,0]
    tn = cm[1,1]
    fp = cm[1,0]
    fn = cm[0,1]

    return tn/(tn+fn)
def FPR(cm):
    tp = cm[0,0]
    tn = cm[1,1]
    fp = cm[1,0]
    fn = cm[0,1]

    return fp/(tn+fp)
def FDR(cm):
    tp = cm[0,0]
    tn = cm[1,1]
    fp = cm[1,0]
    fn = cm[0,1]

    return fp/(tp+fp)
def F1(cm):
    return (2*precision(cm)*recall(cm))/(precision(cm)+recall(cm))
def F2(cm):
    return (5*precision(cm)*recall(cm))/((4*precision(cm))+recall(cm))
def confusion_matrix(preds, y):
    cm = np.zeros((2,2))
    for i in range(y.shape[0]):
        if preds[i] == 1 and y[i] == 1:
            cm[0,0] += 1
        if preds[i] == 0 and y[i] == 1:
            cm[0,1] += 1
        if preds[i] == 1 and y[i] == 0:
            cm[1,0] += 1
        if preds[i] == 0 and y[i] == 0:
            cm[1,1] += 1
    return cm

def getAccuracies(preds, testlabels, learningRate, title):
    print("Accuracies for " , title, " with ", learningRate)
    cm = confusion_matrix(preds,testlabels)
    acc = accuracy(cm)
    prec = precision(cm)
    rcl = recall(cm)
    npv = NPV(cm)
    fpr = FPR(cm)
    fdr = FDR(cm)
    f1 = F1(cm)
    f2 = F2(cm)

    print("Confusion Matrix = ", cm)
    print("Accuracy = ", acc)
    print("Precision = ", prec)
    print("Recall = ", rcl)
    print("Negative Predictive Value = ", npv)
    print("False Positive Rate = ", fpr)
    print("False Discovery Rate = ", fdr)
    print("F1 Value = ", f1)
    print("F2 Value = ", f2)

In [ ]:
# Mini Batch Gradient Ascent
def minibatch_ga(X, Y, batch_size, epochs, learningRate):
    
    samples = X.shape[0]
    features = X.shape[1]

    W = np.random.normal(0, 0.01, (features))
    b = np.random.normal(0, 0.01)
    # Start timer here to time training duration
    start = time.time()
    for i in range(epochs):
        # Divide trainset into batches of size 32 and train
        for k in range(samples // batch_size):

            batch = X[k * batch_size : (k+1) * batch_size]
            label = Y[k * batch_size : (k+1) * batch_size]

            z = np.dot(batch, W) + b
            a = sigmoid(z)

            gradient = np.dot(batch.T, (label-a))/batch_size
            W = W + learningRate * gradient
            b = b + learningRate * (np.sum(label-a)/batch_size)

    # Get duration
    end = time.time()
    print("Training took ", end-start)
    return W, b

W1, b1 = minibatch_ga(trainset, trainlabels, 32, 1000, 10**-4)
preds1 = predict(testset, W1, b1)
getAccuracies(preds1, testlabels, 10**-4, "Mini Batch Gradient Ascent with Batch Size 32")
print("\n" * 3)
W2, b2 = minibatch_ga(trainset, trainlabels, 32, 1000, 10**-3)
preds2 = predict(testset, W2, b2)
getAccuracies(preds2, testlabels, 10**-3, "Mini Batch Gradient Ascent with Batch Size 32")
print("\n" * 3)
W3, b3 = minibatch_ga(trainset, trainlabels, 32, 1000, 10**-2)
preds3 = predict(testset, W3, b3)
getAccuracies(preds3, testlabels, 10**-2, "Mini Batch Gradient Ascent with Batch Size 32")
print("\n" * 3)

Training took  0.5738558769226074
Accuracies for  Mini Batch Gradient Ascent with Batch Size 32  with  0.0001
Confusion Matrix =  [[  8.  61.]
 [  0. 110.]]
Accuracy =  0.659217877094972
Precision =  1.0
Recall =  0.11594202898550725
Negative Predictive Value =  0.6432748538011696
False Positive Rate =  0.0
False Discovery Rate =  0.0
F1 Value =  0.20779220779220778
F2 Value =  0.14084507042253522




Training took  0.5796158313751221
Accuracies for  Mini Batch Gradient Ascent with Batch Size 32  with  0.001
Confusion Matrix =  [[ 49.  20.]
 [ 10. 100.]]
Accuracy =  0.8324022346368715
Precision =  0.8305084745762712
Recall =  0.7101449275362319
Negative Predictive Value =  0.8333333333333334
False Positive Rate =  0.09090909090909091
False Discovery Rate =  0.1694915254237288
F1 Value =  0.7656250000000001
F2 Value =  0.7313432835820896




Training took  0.662522554397583
Accuracies for  Mini Batch Gradient Ascent with Batch Size 32  with  0.01
Confusion Matrix =  [[ 48.  21.]
 [  9. 

In [ ]:
# Full Batch Gradient Ascent
def fullbatch_ga(X, Y, epochs, learningRate):

    samples = X.shape[0]
    features = X.shape[1]

    W = np.random.normal(0, 0.01, (features))
    # W = np.zeros(features)
    b = np.random.normal(0, 0.01)
    # Start timer here to time training duration
    start = time.time()

    for i in range(epochs):
        # print("Epoch => ", i+1)
        if(i%100 == 0 and i > 0):
            print("Weights at epoch ", i, " " , W)
        z = np.dot(X, W) + b
        a = sigmoid(z)

        gradient = np.dot(X.T, (Y-a))/samples
        W = W + learningRate * gradient
        b = b + learningRate * (np.sum(Y-a)/samples)

    # Get duration
    end = time.time()
    print("Training took ", end-start)
    return W,b

W1, b1 = fullbatch_ga(trainset, trainlabels, 1000, 10**-4)
preds1 = predict(testset, W1, b1)
getAccuracies(preds1, testlabels, 10**-4, "Full Batch Gradient Ascent")
print("\n" * 3)
W2, b2 = fullbatch_ga(trainset, trainlabels, 1000, 10**-3)
preds2 = predict(testset, W2, b2)
getAccuracies(preds2, testlabels, 10**-3, "Full Batch Gradient Ascent")
print("\n" * 3)
W3, b3 = fullbatch_ga(trainset, trainlabels, 1000, 10**-2)
preds3 = predict(testset, W3, b3)
getAccuracies(preds3, testlabels, 10**-2, "Full Batch Gradient Ascent")
print("\n" * 3)

Weights at epoch  100   [-0.01512347  0.00969282 -0.01048701 -0.01241675  0.01136307  0.00819097]
Weights at epoch  200   [-0.01656913  0.01047752 -0.01104443 -0.01249236  0.01134338  0.00810724]
Weights at epoch  300   [-0.01801079  0.0112635  -0.01159981 -0.01256765  0.01132397  0.00802453]
Weights at epoch  400   [-0.01944847  0.01205078 -0.01215315 -0.01264264  0.01130485  0.00794281]
Weights at epoch  500   [-0.0208822   0.01283933 -0.01270446 -0.01271732  0.011286    0.0078621 ]
Weights at epoch  600   [-0.02231197  0.01362914 -0.01325374 -0.01279169  0.01126744  0.00778238]
Weights at epoch  700   [-0.02373782  0.01442021 -0.01380102 -0.01286576  0.01124916  0.00770365]
Weights at epoch  800   [-0.02515976  0.01521253 -0.01434629 -0.01293953  0.01123116  0.00762591]
Weights at epoch  900   [-0.0265778   0.01600609 -0.01488956 -0.01301299  0.01121343  0.00754915]
Training took  0.06705212593078613
Accuracies for  Full Batch Gradient Ascent  with  0.0001
Confusion Matrix =  [[ 26.